## Code Review
- 전처리 -> 학습 -> 생성을 프로세스로 인식하고 각각에 대응되는 소스코드를 순서대로 탐색합니다.
- `music_vae_train.py`에서 `data.py`와 관련된 부분을 집중적으로 찾아보면서 전처리 과정을 탐색합니다. (08-20 08:00-09:00, 1 hour)
- `music_vae_train.py`에서 top-down 방식으로 전체적인 학습 과정을 탐색합니다. (08-20 10:00-12:00, 2 hours)
- `music_vae_generate.py`에서 top-down 방식으로 전체적인 생성 과정을 탐색합니다. (08-20 12:00-13:00, 1 hour)
- 과제 수행에 있어서 custom config를 사용해야할 필요성을 느끼고 해당 부분을 다시 확인합니다. (08-20 15:00-17:00, 2 hours)
- 전처리와 학습 과정에서 지나쳤던 부분들을 다시 점검합니다. (08-21 08:00-10:00, 2 hours)

In [ ]:
# Copyright 2022 The Magenta Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<hr style="border-style:dotted">

### 전처리 과정 탐색

In [ ]:
# music_vae_train.py

from magenta.models.music_vae import data
import tensorflow.compat.v1 as tf
import tf_slim

flags = tf.app.flags
FLAGS = flags.FLAGS

def train():
  pass

def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator):
  config = config_map[FLAGS.config]

  if FLAGS.mode == 'train':
    is_training = True
  elif FLAGS.mode == 'eval':
    is_training = False

  def dataset_fn():
    return data.get_dataset(
        config,
        tf_file_reader=tf_file_reader,
        is_training=is_training,
        cache_dataset=FLAGS.cache_dataset)

  if is_training:
    train(
        # ...
        dataset_fn=dataset_fn,
        # ...
        )

- `music_vae_train.py`에서 `data.py`를 호출하는 부분은 `dataset_fn()` 내 `get_dataset()`이 유일하기 때문에,   
  해당 함수를 메인으로 판단하고 시작점으로서 탐색을 수행합니다.

In [ ]:
# data.py

def get_dataset(
    config,
    tf_file_reader=tf.data.TFRecordDataset,
    is_training=False,
    cache_dataset=True):

  batch_size = config.hparams.batch_size
  examples_path = (
      config.train_examples_path if is_training else config.eval_examples_path)
  note_sequence_augmenter = (
      config.note_sequence_augmenter if is_training else None)
  data_converter = config.data_converter
  data_converter.set_mode('train' if is_training else 'eval')

  if examples_path:
    tf.logging.info('Reading examples from file: %s', examples_path)
    num_files = len(tf.gfile.Glob(examples_path))
    if not num_files:
      raise ValueError(
          'No files were found matching examples path: %s' %  examples_path)
    files = tf.data.Dataset.list_files(examples_path)
    dataset = files.interleave(
        tf_file_reader,
        cycle_length=tf.data.experimental.AUTOTUNE,
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
  elif config.tfds_name:
    pass
  # ...

- `get_dataset()` 함수의 파라미터로 `tf.data.TFRecordDataset` 타입의 객체가 요구되는 것으로 보아,   
  MIDI 파일이 아닌, TFRecord 파일로 변환된 데이터를 읽어오는 동작을 짐작해볼 수 있습니다.
- `get_dataset()` 함수에서는 실제로 `config`로부터 `batch_size`, `train_examples_path`, `data_converter`, `mode` 정보를 가져와서,   
  대상 경로가 존재하면 TFRecord 파일을 읽어오고, 존재하지 않으면 `tfds_name`으로 설정된 경로에서 MIDI 파일을 읽어 TFRecord로 변환합니다.

In [ ]:
# music_vae_train.py

def train(config, dataset_fn):
  # ...
  with tf.Graph().as_default():
    # ...
    model = config.model
    # ...
    optimizer = model.train(**_get_input_tensors(dataset_fn(), config))
    # ...

def _get_input_tensors(dataset, config):
  """Get input tensors from dataset."""
  batch_size = config.hparams.batch_size
  iterator = tf.data.make_one_shot_iterator(dataset)
  (input_sequence, output_sequence, control_sequence,
   sequence_length) = iterator.get_next()
  input_sequence.set_shape(
      [batch_size, None, config.data_converter.input_depth])
  output_sequence.set_shape(
      [batch_size, None, config.data_converter.output_depth])
  if not config.data_converter.control_depth:
    control_sequence = None
  else:
    control_sequence.set_shape(
        [batch_size, None, config.data_converter.control_depth])
  sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())

  return {
      'input_sequence': input_sequence,
      'output_sequence': output_sequence,
      'control_sequence': control_sequence,
      'sequence_length': sequence_length
  }

- `TFRecordDataset`을 반환하는 해당 함수는 `music_vae_train` 모듈 내 `train()` 함수의 파라미터로 전달되며,   
  학습 단계에서 `_get_input_tensors()` 함수를 통해 batch size만큼의 시퀀스가 담긴 dictionary를 반환하게 됩니다.

<hr style="border-style:dotted">

### 학습 과정 탐색
- `music_vae_train.py`를 통한 학습 수행을 위해 파이썬 스크립트 실행 과정에서 `mode=train`으로 설정하면,   
  내부적으로 `boolean` 타입의 `is_training` 변수 값을 설정해 `train()` 함수를 호출할지 `evaluate()` 함수를 호출할지 결정합니다.
- `train()` 함수는 `config`에서 `run_dir`로 설정된 디렉토리 아래 `train` 위치를 `train_dir`로서 전달받고,   
  그 외에 전체 config 정보, 데이터셋을 불러오는 함수 `dataset_fn()`, 기타 FLAGS 설정을 파라미터로 받습니다.

In [ ]:
# configs.py

import collections
from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import data
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae.base_model import MusicVAE

HParams = contrib_training.HParams


class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

  def values(self):
    return self._asdict()

CONFIG_MAP = {}

CONFIG_MAP['cat-drums_2bar_small'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.CategoricalLstmDecoder()),
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512,
            # ...
        )),
    note_sequence_augmenter=None,
    data_converter=data.DrumsConverter(
        max_bars=100,  # Truncate long drum sequences before slicing.
        slice_bars=2,
        steps_per_quarter=4,
        roll_input=True),
    train_examples_path=None,
    eval_examples_path=None,
)

- `train()` 함수를 알아보기에 앞서, `config`에 어떤 정보가 담겨있는지 보기 위해 `configs.py`를 탐색하기로 했고,   
  4마디에 해당하는 드럼 샘플을 추출하기 위한 목적과 가장 유사한 설정 정보인 `cat-drums_2bar_small`를 중심으로 알아봅니다.
- 해당하는 config에 저장된 것에는 `model`, `hparams`, `data_converter`, `train_examples_path` 등이 있는데,   
  `hparams`는 이름에서 보듯이 하이퍼파라미터 설정이고 `train_examples_path`는 TFRecord 파일에 대한 경로임을 앞선 과정을 통해 알 수 있습니다.
- 다만, 해당 설정에서 보이는 모델의 구조는 계층적 구조로 이루어지지 않은,   
  위 논문에서 정의된 flat baseline에 해당하는 모델로 예상되기 때문에 계층적 디코더 모델에 대한 설정을 같이 확인해 봅니다.

In [ ]:
# data.py

import note_seq
from note_seq import drums_encoder_decoder
import numpy as np

REDUCED_DRUM_PITCH_CLASSES = drums_encoder_decoder.DEFAULT_DRUM_TYPE_PITCHES
OUTPUT_VELOCITY = 80


class BaseNoteSequenceConverter(object):
    pass


class DrumsConverter(BaseNoteSequenceConverter):
    """Converter for legacy drums with either pianoroll or one-hot tensors."""

    def __init__(self, max_bars=None, slice_bars=None, pitch_classes=None,):
        self._pitch_classes = pitch_classes or REDUCED_DRUM_PITCH_CLASSES
        self._pitch_class_map = {}
        # ...
        num_classes = len(self._pitch_classes)
        # ...
        self._oh_encoder_decoder = note_seq.MultiDrumOneHotEncoding(
            drum_type_pitches=[(i,) for i in range(num_classes)])

    def from_tensors(self, samples, unused_controls=None):
        output_sequences = []
        for s in samples:
            # ...
            s = np.argmax(s, axis=-1)
            if self.end_token is not None and self.end_token in s:
                s = s[:s.tolist().index(self.end_token)]
                events_list = [self._oh_encoder_decoder.decode_event(e) for e in s]
        events_list = [
            frozenset(self._pitch_classes[c][0] for c in e) for e in events_list]
        track = note_seq.DrumTrack(
            events=events_list,
            steps_per_bar=self._steps_per_bar,
            steps_per_quarter=self._steps_per_quarter)
        output_sequences.append(track.to_sequence(velocity=OUTPUT_VELOCITY))
        return output_sequences

- 모델을 알아보기 앞서 `data_converter`로 전달되는 `DrumsConverter`가 어떤 동작을 하는지 확인해보았습니다.
- `Magenta`에서 MIDI 데이터를 처리하는 부분은 대부분이 `NoteSequence`라는 외부 모듈에 담겨있는데,   
  그 중에서 드럼 데이터를 추출하는 것으로 예상되는 `MultiDrumOneHotEncoding`과 `decode_event` 메서드를 알아볼 필요가 있습니다.

In [ ]:
# note_seq/drums_encoder_decoder.py

from note_seq import encoder_decoder


class MultiDrumOneHotEncoding(encoder_decoder.OneHotEncoding):

  def __init__(self, drum_type_pitches=None, ignore_unknown_drums=True):
    self._drum_map = dict(enumerate(drum_type_pitches))
    self._inverse_drum_map = dict((pitch, index)
                                  for index, pitches in self._drum_map.items()
                                  for pitch in pitches)
    self._ignore_unknown_drums = ignore_unknown_drums

  def decode_event(self, index):
    bits = reversed(str(bin(index)))
    # Use the first "pitch" for each drum type.
    return frozenset(self._drum_map[i][0]
                     for i, b in enumerate(bits)
                     if b == '1')

- `DrumsConverter`에서 `MultiDrumOneHotEncoding`를 생성할 때 클래스에 대한 2차원 배열을 전달하는데,   
  해당 배열을 인덱스를 키값으로 하는 딕셔너리로 변환해 저장합니다.
- `from_tensors()`에서 `events_list`를 생성하는 과정에서 `decode_event`가 호출되는데,   
  각각의 이벤트 시퀀스에 대해 인덱스와 대응되는 클래스를 반환하는 과정이 진행되는 것이라 판단됩니다.

In [ ]:
# configs.py

CONFIG_MAP['hierdec-trio_16bar'] = Config(
    model=MusicVAE(
        lstm_models.BidirectionalLstmEncoder(),
        lstm_models.HierarchicalLstmDecoder(
            lstm_models.SplitMultiOutLstmDecoder(
                core_decoders=[
                    lstm_models.CategoricalLstmDecoder(),
                    lstm_models.CategoricalLstmDecoder(),
                    lstm_models.CategoricalLstmDecoder()],
                output_depths=[
                    90,  # melody
                    90,  # bass
                    512,  # drums
                ]),
            level_lengths=[16, 16],
            disable_autoregression=True)),
    # ...
)

- 모델에 대해 알아보기 위해 다시 config로 돌아와서, 16마디의 3채널 데이터에 대한 계층적 디코더 모델의 설정을 확인했을 때,   
  `HierarchicalLstmDecoder`의 하위 계층 요소로 `CategoricalLstmDecoder`를 넣으면 이상적인 설정이 될 것이라 예상합니다.

In [ ]:
# base_model.py

class MusicVAE(object):
  """Music Variational Autoencoder."""

  def __init__(self, encoder, decoder):
    """Initializer for a MusicVAE model."""
    self._encoder = encoder
    self._decoder = decoder

- `model`로 전달되는 객체인 `MusicVAE`는 `base_model.py`에서 확인할 수 있듯이 `encoder`와 `decoder`로 구성되며,   
  각각 `BidirectionalLstmEncoder`, `HierarchicalLstmDecoder`를 사용합니다.

In [ ]:
# lstm_models.py

import magenta.contrib.rnn as contrib_rnn
import magenta.contrib.seq2seq as contrib_seq2seq
from magenta.models.music_vae import base_model
from magenta.models.music_vae import lstm_utils
import tensorflow_probability as tfp


class BidirectionalLstmEncoder(base_model.BaseEncoder):
  """Bidirectional LSTM Encoder."""

  def build(self, hparams, is_training=True, name_or_scope='encoder'):
    self._is_training = is_training
    self._name_or_scope = name_or_scope
    # ...
    self._cells = lstm_utils.build_bidirectional_lstm(
        layer_sizes=hparams.enc_rnn_size,
        dropout_keep_prob=hparams.dropout_keep_prob,
        residual=hparams.residual_encoder,
        is_training=is_training)

  def encode(self, sequence, sequence_length):
    cells_fw, cells_bw = self._cells

    _, states_fw, states_bw = contrib_rnn.stack_bidirectional_dynamic_rnn(
        cells_fw,
        cells_bw,
        sequence,
        sequence_length=sequence_length,
        time_major=False,
        dtype=tf.float32,
        scope=self._name_or_scope)
    last_h_fw = states_fw[-1][-1].h
    last_h_bw = states_bw[-1][-1].h

    return tf.concat([last_h_fw, last_h_bw], 1)

In [ ]:
# lstm_utils.py

rnn = tf.nn.rnn_cell

def rnn_cell(rnn_cell_size, dropout_keep_prob, residual, is_training=True):
  """Builds an LSTMBlockCell based on the given parameters."""
  dropout_keep_prob = dropout_keep_prob if is_training else 1.0
  cells = []
  for i in range(len(rnn_cell_size)):
    cell = contrib_rnn.LSTMBlockCell(rnn_cell_size[i])
    # ...
    cells.append(cell)
  return rnn.MultiRNNCell(cells)

def build_bidirectional_lstm(
    layer_sizes, dropout_keep_prob, residual, is_training):
  """Build the Tensorflow graph for a bidirectional LSTM."""

  cells_fw = []
  cells_bw = []
  for layer_size in layer_sizes:
    cells_fw.append(
        rnn_cell([layer_size], dropout_keep_prob, residual, is_training))
    cells_bw.append(
        rnn_cell([layer_size], dropout_keep_prob, residual, is_training))

  return cells_fw, cells_bw

- `BidirectionalLstmEncoder`는 `build_bidirectional_lstm()` 함수를 통해 모델 구조를 생성하며,   
  해당 함수는 내부적으로 forward, backward 방향에 대한 두 가지 RNN cell을 반환합니다.
- 각각의 RNN cell은 RNN 구조를 상속받는 LSTM block으로 구성되며,   
  하이퍼파라미터로 전달한 `enc_rnn_size`만큼의 초기화된 가중치를 가지고 있습니다.
- 학습 과정에서 실행되는 `encode()` 구문에서는 양방향 RNN에서 각 방향의 마지막 hidden state를 병합한 결과를 반환합니다.

In [ ]:
# base_model.py

ds = tfp.distributions


class MusicVAE(object):
  def encode(self, sequence, sequence_length, control_sequence=None):
    """Encodes input sequences into a MultivariateNormalDiag distribution."""
    hparams = self.hparams
    z_size = hparams.z_size
    sequence = tf.to_float(sequence)
    #...
    encoder_output = self.encoder.encode(sequence, sequence_length)

    mu = tf.layers.dense(
        encoder_output,
        z_size,
        name='encoder/mu',
        kernel_initializer=tf.random_normal_initializer(stddev=0.001))
    sigma = tf.layers.dense(
        encoder_output,
        z_size,
        activation=tf.nn.softplus,
        name='encoder/sigma',
        kernel_initializer=tf.random_normal_initializer(stddev=0.001))

    return ds.MultivariateNormalDiag(loc=mu, scale_diag=sigma)

- 위 인코딩 과정은 `MusicVAE`의 `encode()` 메서드에서 `encoder_output`을 생성하기 위한 중간 과정이며,   
  이후 해당 결과를 각각의 Dense 레이어를 거치게 하여 $\mu$와 $\sigma$를 생성합니다.
- Reparameterization Trick이 적용된 별도 모듈을 사용해 $\mu$와 $\sigma$에 해당하는 정규 분포를 생성 및 반환합니다.

In [ ]:
# lstm_models.py

class HierarchicalLstmDecoder(base_model.BaseDecoder):
  """Hierarchical LSTM decoder."""

  def build(self, hparams, output_depth, is_training=True):
    self.hparams = hparams
    self._output_depth = output_depth
    self._total_length = hparams.max_seq_len
    #...
    self._hier_cells = [
        lstm_utils.rnn_cell(
            hparams.dec_rnn_size,
            dropout_keep_prob=hparams.dropout_keep_prob,
            residual=hparams.residual_decoder)
        # Subtract 1 for the core decoder level
        for _ in range(len(self._level_lengths) - 1)]

    with tf.variable_scope('core_decoder', reuse=tf.AUTO_REUSE):
      self._core_decoder.build(hparams, output_depth, is_training)

- `HierarchicalLstmDecoder`는 하이퍼파라미터로 전달한 `level_lengths`개만큼의 LSTM block을 리스트로 가집니다.
- 각각의 LSTM block은 마찬가지로 하이퍼파라미터로 전달한 `dec_rnn_size`만큼의 state로 구성되어 있습니다.

In [ ]:
# lstm_models.py

class BaseLstmDecoder(base_model.BaseDecoder):
  """Abstract LSTM Decoder class."""

  def build(self, hparams, output_depth, is_training=True):
    # ...
    self._output_layer = tf.layers.Dense(
        output_depth, name='output_projection')
    self._dec_cell = lstm_utils.rnn_cell(
        hparams.dec_rnn_size, hparams.dropout_keep_prob,
        hparams.residual_decoder, is_training)

  def _decode(self, z, helper, input_shape, max_length=None):
    initial_state = lstm_utils.initial_cell_state_from_embedding(
        self._dec_cell, z, name='decoder/z_to_initial_state')

    decoder = lstm_utils.Seq2SeqLstmDecoder(
        self._dec_cell,
        helper,
        initial_state=initial_state,
        input_shape=input_shape,
        output_layer=self._output_layer)
    final_output, final_state, final_lengths = contrib_seq2seq.dynamic_decode(
        decoder,
        maximum_iterations=max_length,
        swap_memory=True,
        scope='decoder')
    results = lstm_utils.LstmDecodeResults(
        rnn_input=final_output.rnn_input[:, :, :self._output_depth],
        rnn_output=final_output.rnn_output,
        samples=final_output.sample_id,
        final_state=final_state,
        final_sequence_lengths=final_lengths)

    return results

- `CategoricalLstmDecoder`는 base model을 상속받는 `HierarchicalLstmDecoder`와 다르게   
  `BaseLstmDecoder`라는 추상 클래스를 상속받기 때문에, 해당하는 모델을 우선적으로 탐색했습니다.
- `BaseLstmDecoder`는 내부적으로 단방향 LSTM block과 Dense 레이어로 구성된 구조입니다.
- `BaseLstmDecoder`의 `decode()` 기능은 `Seq2Seq` 디코더의 동작 방식을 활용한 것으로 추정되며,   
  해당 `dynamic_decode()` 함수는 `max_length`에 해당하는 `maximum_iterations`까지 반복문을 수행하면서 연속된 state를 생성합니다.

In [ ]:
# lstm_models.py

class CategoricalLstmDecoder(BaseLstmDecoder):
  """LSTM decoder with single categorical output."""

  def _flat_reconstruction_loss(self, flat_x_target, flat_rnn_output):
    flat_logits = flat_rnn_output
    flat_truth = tf.argmax(flat_x_target, axis=1)
    flat_predictions = tf.argmax(flat_logits, axis=1)
    r_loss = tf.nn.softmax_cross_entropy_with_logits(
        labels=flat_x_target, logits=flat_logits)

    metric_map = {
        'metrics/accuracy':
            tf.metrics.accuracy(flat_truth, flat_predictions),
        'metrics/mean_per_class_accuracy':
            tf.metrics.mean_per_class_accuracy(
                flat_truth, flat_predictions, int(flat_x_target.shape[-1])),
    }
    return r_loss, metric_map

  def _sample(self, rnn_output, temperature=1.0):
    sampler = tfp.distributions.OneHotCategorical(
        logits=rnn_output / temperature, dtype=tf.float32)
    return sampler.sample()

- `CategoricalLstmDecoder`는 `BaseLstmDecoder`의 구조를 그대로 가져오면서,   
  베르누이 분포를 따르는 $p_\theta$에 대한 reconstruction error인 cross entropy를 loss로 사용함을 알 수 있습니다.

In [ ]:
# lstm_models.py

class HierarchicalLstmDecoder(base_model.BaseDecoder):
  """Hierarchical LSTM decoder."""

  def _hierarchical_decode(self, z, base_decode_fn):
    """Depth first decoding from `z`, passing final embeddings to base fn."""
    batch_size = z.shape[0]
    # Subtract 1 for the core decoder level.
    num_levels = len(self._level_lengths) - 1

    hparams = self.hparams
    batch_size = hparams.batch_size

    def recursive_decode(initial_input, path=None):
      """Recursive hierarchical decode function."""
      path = path or []
      level = len(path)

      if level == num_levels:
        with tf.variable_scope('core_decoder', reuse=tf.AUTO_REUSE):
          return base_decode_fn(initial_input, path)

      scope = tf.VariableScope(
          tf.AUTO_REUSE, 'decoder/hierarchical_level_%d' % level)
      num_steps = self._level_lengths[level]
      with tf.variable_scope(scope):
        state = lstm_utils.initial_cell_state_from_embedding(
            self._hier_cells[level], initial_input, name='initial_state')
      if level not in self._disable_autoregression:
        # The initial input should be the same size as the tensors returned by
        # next level.
        if self._hierarchical_encoder:
          input_size = self._hierarchical_encoder.level(0).output_depth
        elif level == num_levels - 1:
          input_size = sum(tf.nest.flatten(self._core_decoder.state_size))
        else:
          input_size = sum(
              tf.nest.flatten(self._hier_cells[level + 1].state_size))
        next_input = tf.zeros([batch_size, input_size])
      lower_level_embeddings = []
      for i in range(num_steps):
        if level in self._disable_autoregression:
          next_input = tf.zeros([batch_size, 1])
        else:
          next_input = tf.concat([next_input, initial_input], axis=1)
        with tf.variable_scope(scope):
          output, state = self._hier_cells[level](next_input, state, scope)
        next_input = recursive_decode(output, path + [i])
        lower_level_embeddings.append(next_input)
      if self._hierarchical_encoder:
        # Return the encoding of the outputs using the appropriate level of the
        # hierarchical encoder.
        enc_level = num_levels - level
        return self._hierarchical_encoder.level(enc_level).encode(
            sequence=tf.stack(lower_level_embeddings, axis=1),
            sequence_length=tf.fill([batch_size], num_steps))
      else:
        # Return the final state.
        return tf.concat(tf.nest.flatten(state), axis=-1)

    return recursive_decode(z)

- `HierarchicalLstmDecoder`의 전체 디코딩 동작을 확인했을 때,   
  core decoder에서부터 재귀적으로 모든 계층의 디코더를 순회하면서 상위 계층의 $z$로부터 도출되는 state를 모두 종합하여 반환하는 것으로 보입니다.
- 위 `hierdec-trio_16bar` 설정에서 4마디를 출력하게 하려면 가장 아랫 계층에 대한 `level_lengths`를 4로 설정해야할 것입니다.

In [ ]:
# music_vae_train.py

def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):
  """Train loop."""
  # ...
  with tf.Graph().as_default():
    with tf.device(tf.train.replica_device_setter(
        num_ps_tasks, merge_devices=True)):
      model = config.model
      model.build(config.hparams,
                  config.data_converter.output_depth,
                  is_training=True)

      optimizer = model.train(**_get_input_tensors(dataset_fn(), config))
      # ...
      grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
      global_norm = tf.global_norm(grads)
      tf.summary.scalar('global_norm', global_norm)

      if config.hparams.clip_mode == 'value':
        g = config.hparams.grad_clip
        clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
      # ...
      train_op = optimizer.apply_gradients(
          list(zip(clipped_grads, var_list)),
          global_step=model.global_step,
          name='train_step')

      logging_dict = {'global_step': model.global_step,
                      'loss': model.loss}
      #...
      scaffold = tf.train.Scaffold()
      tf_slim.training.train()

- 원래 목표로 했던 `train()` 함수로 돌아와, 시작 부분에서 config에 포함된 모델 정보를 바탕으로   
  `hparams`와 `data_converter`를 바탕으로 인코더-디코더 구조의 MusicVAE 모델을 생성함을 알 수 있습니다.
- 또한, 전처리 과정에서 확인한 `_get_input_tensors()` 함수를 통해 batch size만큼의 데이터를 가져와 학습 데이터로 사용합니다.
- optimzer는 reconstruction error인 cross entropy를 최소화하는 방향으로 경사하강법을 수행하면서 최적화를 진행합니다.
- Scaffold와 tf_slim의 기능은 완전히 이해하지는 못했지만, 아래 참고 자료를 바탕으로 분산 처리를 위한 구문임을 짐작해 볼 수 있습니다.
- 학습 후 평가를 위한 `evaluate()` 함수는 `train()` 함수와 동일한 구조에서 모델 학습에 대한 부분만 제거되었습니다.

### References
- [Distributed TensorFlow (TensorFlow Dev Summit 2017)](https://youtu.be/la_M6bCV91M)
- https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/Scaffold
- https://digitalbourgeois.tistory.com/51

<hr style="border-style:dotted">

### 생성 과정 탐색
- `music_vae_generate.py`를 실행하기 위해선 `run_dir` 또는 `checkpoint_file`, `output_dir`, `mode`를 설정해야 합니다.
- `mode`에는 `sample`과 `interpolate`가 있고 샘플 생성을 위해서는 `sample`을 설정합니다.

In [ ]:
# music_vae_generate.py

import os
import time
from magenta.models.music_vae import TrainedModel
import note_seq

flags = tf.app.flags
logging = tf.logging
FLAGS = flags.FLAGS

def run(config_map):
  # ...
  config = config_map[FLAGS.config]
  # ...
  logging.info('Loading model...')
  if FLAGS.run_dir:
    checkpoint_dir_or_path = os.path.expanduser(
        os.path.join(FLAGS.run_dir, 'train'))
  else:
    checkpoint_dir_or_path = os.path.expanduser(FLAGS.checkpoint_file)
  model = TrainedModel(
      config, batch_size=min(FLAGS.max_batch_size, FLAGS.num_outputs),
      checkpoint_dir_or_path=checkpoint_dir_or_path)
  # ...

- 생성 과정은 단순하게 모델을 불러와서 샘플링 결과를 저장하는 것입니다.
- `run()` 함수의 시작 부분에선 config를 검증하고 `TrainedModel`을 생성합니다.

In [ ]:
# trained_model.py

import copy

class TrainedModel(object):
  def __init__(self, config, batch_size, checkpoint_dir_or_path=None,
               var_name_substitutions=None, session_target='', **sample_kwargs):
    if tf.gfile.IsDirectory(checkpoint_dir_or_path):
      checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir_or_path)
    else:
      checkpoint_path = checkpoint_dir_or_path
    self._config = copy.deepcopy(config)
    self._config.data_converter.set_mode('infer')
    self._config.hparams.batch_size = batch_size
    with tf.Graph().as_default():
      model = self._config.model
      model.build(
          self._config.hparams,
          self._config.data_converter.output_depth,
          is_training=False)
      # Input placeholders
      self._temperature = tf.placeholder(tf.float32, shape=())
      # ...

- `TrainedModel`의 내부 구조를 알아보기 위해 우선 초기화 메서드를 확인했을 때,   
  체크포인트 경로를 받고 `trainable=False` 설정과 함께 모델을 불러와 저장하는 것을 알 수 있습니다.

In [ ]:
# trained_model.py

import numpy as np

class TrainedModel(object):
  def sample(self, n=None, length=None, temperature=1.0, same_z=False,
             c_input=None):
    """Generates random samples from the model."""
    batch_size = self._config.hparams.batch_size
    n = n or batch_size
    z_size = self._config.hparams.z_size

    if not length and self._config.data_converter.end_token is None:
      raise ValueError(
          'A length must be specified when the end token is not used.')
    length = length or tf.int32.max

    feed_dict = {
        self._temperature: temperature,
        self._max_length: length
    }

    if self._z_input is not None and same_z:
      z = np.random.randn(z_size).astype(np.float32)
      z = np.tile(z, (batch_size, 1))
      feed_dict[self._z_input] = z

    if self._c_input is not None:
      feed_dict[self._c_input] = c_input

    outputs = []
    for _ in range(int(np.ceil(n / batch_size))):
      if self._z_input is not None and not same_z:
        feed_dict[self._z_input] = (
            np.random.randn(batch_size, z_size).astype(np.float32))
      outputs.append(self._sess.run(self._outputs, feed_dict))
    samples = np.vstack(outputs)[:n]
    if self._c_input is not None:
      return self._config.data_converter.from_tensors(
          samples, np.tile(np.expand_dims(c_input, 0), [batch_size, 1, 1]))
    else:
      return self._config.data_converter.from_tensors(samples)

- 샘플을 생성하는 `sample()` 메서드는 샘플 수 `n`, 샘플 별 최대 길이 `length`, softmax 함수에서의 온도계수 `temperature` 등을 받아,   
  `NoteSequence` 타입의 샘플 객체 리스트를 반환합니다.
- `z`를 별도로 지정하지 않을 경우 랜덤한 `z`를 생성하고, 전체 샘플 수에 대해 batch size 단위로 반복하며 샘플을 생성합니다.

In [ ]:
# music_vae_generate.py

def run(config_map):
  date_and_time = time.strftime('%Y-%m-%d_%H%M%S')
  # ...
  config = config_map[FLAGS.config]
  # ...
  model = TrainedModel()
  # ...
  if FLAGS.mode == 'interpolate':
    pass
  elif FLAGS.mode == 'sample':
    logging.info('Sampling...')
    results = model.sample(
        n=FLAGS.num_outputs,
        length=config.hparams.max_seq_len,
        temperature=FLAGS.temperature)

  basename = os.path.join(
      FLAGS.output_dir,
      '%s_%s_%s-*-of-%03d.mid' %
      (FLAGS.config, FLAGS.mode, date_and_time, FLAGS.num_outputs))
  logging.info('Outputting %d files as `%s`...', FLAGS.num_outputs, basename)
  for i, ns in enumerate(results):
    note_seq.sequence_proto_to_midi_file(ns, basename.replace('*', '%03d' % i))

  logging.info('Done.')

- 다시 `run()` 함수로 돌아와서, 위와 같은 과정을 거쳐 생성된 샘플이 `results`에 담기고,   
  `output_dir`에 MIDI 파일이 저장되는 것을 확인할 수 있습니다.